In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

# import

In [ ]:
# export 
import sys 
if '..' not in sys.path:
    sys.path.append('..')
from exp import resnet_ssd

In [ ]:
# export
from FLAI.detect_symbol.exp import databunch as databunch_detsym
from FLAI.detect_symbol.exp import init_model as init_model_detsym

In [ ]:
# export
from fastai.basic_train import Learner,DatasetType

In [ ]:
# export
from fastai.callbacks.hooks import Hook,Hooks

In [ ]:
# export
from torch import nn

In [ ]:
# export
import torch

In [ ]:
from IPython.core import debugger as idb

In [ ]:
# export
from fastprogress.fastprogress import progress_bar

In [ ]:
# export
import re

# functions

# test

In [ ]:
device = torch.device('cuda')

In [ ]:
#先用detect_symbol的数据集测试
data = databunch_detsym.get_databunch(data_root = '../../detect_symbol/data/ds_20200429', bs=8, device=device)
x,y = data.one_batch()

### resnet18_ssd

In [ ]:
model = resnet_ssd.get_resnet18_1ssd(num_classes = 17)

In [ ]:
# check layer output stats before runtime init.
init_model_detsym.show_layer_stats(model,x)

In [ ]:
# runtime init
init_model_detsym.runtime_init_linear(model, x, init_model_detsym.hook_init);

In [ ]:
# check layer output stats after runtime init
init_model_detsym.show_layer_stats(model,x)

In [ ]:
# 因为runtime_init耗时较长，保存初始化后的模型，方便快速调用
# torch.save(model.state_dict(), './models/resnet18_ssd_init.pth') # 只在要保存初始化模型时运行该行代码

### resnet18_1ssd

In [ ]:
#符号检测的17种符号！！！
model = resnet_ssd.get_resnet18_1ssd(num_classes = 17)

#### LSUV

In [ ]:
# check layer output stats before runtime init.
init_model_detsym.show_layer_stats(model,x)

In [ ]:
# runtime init
init_model_detsym.runtime_init_linear(model, x, init_model_detsym.hook_init);

In [ ]:
# check layer output stats after runtime init
init_model_detsym.show_layer_stats(model,x)

In [ ]:
# 因为runtime_init耗时较长，保存初始化后的模型，方便快速调用
# torch.save(model.state_dict(), './models/resnet18_1ssd_init.pth') # 只在要保存初始化模型时运行该行代码

#### init_to_pretrained

In [ ]:
pdict = torch.load('../../detect_symbol/models/pytorch_pretrained/resnet18-5c106cde.pth')

In [ ]:
# 将模型中与 resnet18 对应的部分的参数从预训练模型加载
link_names = [(r'^conv1',        'conv1'),
               (r'^bn1',          'bn1'),
               (r'^res_blocks.0', 'layer1'),
               (r'^res_blocks.1', 'layer2'),
               (r'^res_blocks.2', 'layer3'),
               (r'^res_blocks.3', 'layer4')]
init_model_detsym.init_to_pretrained(model, pdict, link_names)

In [ ]:
# check layer output stats before runtime init.
init_model_detsym.show_layer_stats(model,x)

In [ ]:
# torch.save(model.state_dict(), './models/pretrained_res18_1ssd.pth') # 只在要保存初始化模型时运行该行代码

## 测试加载之前detect_symbol的训练状态

In [ ]:
fn = '../../detect_symbol/data/ds_20200429/models/run_0.pth'
fn = '../../detect_symbol_bak20200427/data/ds_20200227/models/run_0.pth'
#../detect_symbol/run_log/20200429/run_0.pth
#fn = '../detect_symbol/models/resnet18_1ssd_2_init.pth'
dic = torch.load(fn)

In [ ]:
model2 = resnet_ssd.get_resnet18_1ssd(num_classes = 17)

In [ ]:
for n, p in model2.named_parameters():
    assert p.data.shape == dic['model'][n].data.shape, '%s:%s!=%s' % (n,p.data.shape, dic['model'][n].data.shape)
    print(n, p.data.shape, dic['model'][n].data.shape)
    p.data = dic['model'][n].data.to(torch.device('cpu'))
    #print(dic['model'][n].shape, p.sum().cpu().detach().numpy())
    print(n, p.data.shape, dic['model'][n].data.shape, '====')
    
    

# export

In [45]:
!python ../notebook2script.py --fname 'init_model.ipynb' --outputDir '../exp/'

Converted init_model.ipynb to ../exp/init_model.py
